In [ ]:
import os
import gc
import optuna
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from optuna.integration import XGBoostPruningCallback
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")


def configure_gpu():
    try:
        X_demo = np.random.rand(500, 20).astype(np.float32)
        y_demo = np.random.randint(0, 2, 500)
        dtrain = xgb.DMatrix(X_demo, label=y_demo)
        xgb.train(
            params={"objective": "binary:logistic", "device": "cuda", "tree_method": "hist"},
            dtrain=dtrain,
            num_boost_round=1
        )
        print("GPU acceleration enabled")
        return True
    except:
        print("GPU not available, using CPU.")
        return False

USE_XGB_GPU = configure_gpu()

def reduce_memory_usage(df):
    start = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtype
        if col_type in ["int64", "int32"]:
            c_min, c_max = df[col].min(), df[col].max()
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            else:
                df[col] = df[col].astype(np.int32)
        elif col_type == "float64":
            df[col] = df[col].astype(np.float32)
    end = df.memory_usage().sum() / 1024**2
    print(f"Memory optimized: {start:.2f} MB → {end:.2f} MB")
    return df


TRAIN_PATH = "/content/train.csv"
TEST_PATH = "/content/test.csv"
TARGET_COL = "WeightCategory"
ID_COL = "id"
N_TRIALS = 60
N_SPLITS = 5


train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)
train = reduce_memory_usage(train)
test = reduce_memory_usage(test)


def add_bmi_category(df):
    if "Weight" in df.columns and "Height" in df.columns:
        df["BMI"] = df["Weight"] / ((df["Height"] / 100) ** 2)
        conditions = [
            (df["BMI"] < 18.5),
            (df["BMI"] >= 18.5) & (df["BMI"] < 25),
            (df["BMI"] >= 25) & (df["BMI"] < 30),
            (df["BMI"] >= 30) & (df["BMI"] < 35),
            (df["BMI"] >= 35)
        ]
        categories = ["Underweight", "Normal", "Overweight", "Obese", "Extremely Obese"]
        df["BMI_Category"] = np.select(conditions, categories, default='Unknown') # Set default to a string
        df.drop(columns=["BMI"], inplace=True)
    return df


train = add_bmi_category(train)
test = add_bmi_category(test)


X = train.drop(columns=[TARGET_COL, ID_COL], errors="ignore")
y = train[TARGET_COL]
test_features = test.drop(columns=[ID_COL], errors="ignore")


for col in set(X.columns) - set(test_features.columns):
    test_features[col] = np.nan
test_features = test_features[X.columns]

cat_cols = X.select_dtypes(include=["object", "category"]).columns
for col in cat_cols:
    X[col] = X[col].astype("category")
    test_features[col] = test_features[col].astype("category")

label_enc = LabelEncoder()
y_enc = label_enc.fit_transform(y)
num_classes = len(label_enc.classes_)
print(f"Data prepared → {X.shape[1]} features, {num_classes} classes")


def objective(trial):
    params = {
        "objective": "multi:softprob",
        "num_class": num_classes,
        "eval_metric": "mlogloss",
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 0.0, 0.5),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.5, 3.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
        "tree_method": "hist",
        "device": "cuda" if USE_XGB_GPU else "cpu",
        "seed": 42,
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accs = []

    for tr_idx, va_idx in skf.split(X, y_enc):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y_enc[tr_idx], y_enc[va_idx]
        dtrain = xgb.DMatrix(X_tr, label=y_tr, enable_categorical=True)
        dvalid = xgb.DMatrix(X_va, label=y_va, enable_categorical=True)

        model = xgb.train(
            params=params,
            dtrain=dtrain,
            evals=[(dvalid, "val")],
            num_boost_round=1500,
            early_stopping_rounds=100,
            verbose_eval=False,
            callbacks=[XGBoostPruningCallback(trial, "val-mlogloss")]
        )

        preds = model.predict(dvalid)
        accs.append(accuracy_score(y_va, preds.argmax(axis=1)))

        del model, dtrain, dvalid
        gc.collect()

    return np.mean(accs)


print("Running Optuna search...")
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=N_TRIALS)
best_params = study.best_params
print("\nBest Optuna Parameters:")
print(best_params)

GPU acceleration enabled


[I 2025-10-26 11:22:46,472] A new study created in memory with name: no-name-61f77f9e-16dd-47fc-8a41-b583c8330d35


Memory optimized: 2.13 MB → 1.57 MB
Memory optimized: 0.68 MB → 0.49 MB
Data prepared → 17 features, 7 classes
Running Optuna search...


[I 2025-10-26 11:25:43,532] Trial 0 finished with value: 0.9058137835944052 and parameters: {'learning_rate': 0.010879887248903005, 'max_depth': 9, 'min_child_weight': 10, 'subsample': 0.6610541989055381, 'colsample_bytree': 0.693983470867179, 'gamma': 0.3617694776687417, 'reg_lambda': 0.5620086023734077, 'reg_alpha': 0.7686313645452107}. Best is trial 0 with value: 0.9058137835944052.
[I 2025-10-26 11:27:44,080] Trial 1 finished with value: 0.9048478903649219 and parameters: {'learning_rate': 0.017645551606793795, 'max_depth': 6, 'min_child_weight': 3, 'subsample': 0.7803987458625848, 'colsample_bytree': 0.9174113028701733, 'gamma': 0.39598360303052976, 'reg_lambda': 2.771386624380996, 'reg_alpha': 0.8003422132136836}. Best is trial 0 with value: 0.9058137835944052.
[I 2025-10-26 11:28:04,319] Trial 2 finished with value: 0.9030453221243351 and parameters: {'learning_rate': 0.19232906307344733, 'max_depth': 6, 'min_child_weight': 3, 'subsample': 0.7188951703647674, 'colsample_bytree':


Best Optuna Parameters:
{'learning_rate': 0.011094146426713617, 'max_depth': 9, 'min_child_weight': 5, 'subsample': 0.7377019359742616, 'colsample_bytree': 0.6464392505901457, 'gamma': 0.4901785320126722, 'reg_lambda': 2.500928677232875, 'reg_alpha': 0.9107966107743035}
